In [1]:
import pandas as pd
import numpy as np

In [2]:
gws_dataset=pd.read_csv("merged_gw.csv")
gws_dataset=gws_dataset[['name','position','expected_goals','expected_assists','transfers_in','transfers_out','expected_goals_conceded','total_points']]

In [3]:
gws_dataset['expected_goals'] = gws_dataset.expected_goals.astype(float)
gws_dataset['expected_assists'] = gws_dataset.expected_assists.astype(float)
gws_dataset['expected_goals_conceded'] = gws_dataset.expected_goals_conceded.astype(float)

In [5]:
fwd_gws_dataset= gws_dataset.loc[gws_dataset.position == 'FWD']
fw_X=fwd_gws_dataset.iloc[:,2:6]
fw_Y=fwd_gws_dataset.iloc[:,-1].values


In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
fw_X=scaler.fit_transform(fw_X)

In [9]:
from sklearn.model_selection import train_test_split
fw_X_train , fw_X_test , fw_Y_train , fw_Y_test = train_test_split(fw_X,fw_Y,test_size=0.2)

In [14]:
from sklearn.linear_model import LinearRegression
linear_regressor=LinearRegression()
linear_regressor.fit(fw_X_train,fw_Y_train)

LinearRegression()

In [15]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Define the kernel (covariance function)
kernel = 1.0 * RBF(length_scale=1.0)

# Create the Gaussian Process Regressor model
gp_regressor = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

gp_regressor.fit(fw_X_train,fw_Y_train)

c:\Users\anasn\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1),
                         n_restarts_optimizer=10)

In [36]:
from sklearn.svm import SVR
# Create an SVR model with a linear kernel
svr_regressor = SVR(kernel='linear', C=1.2, epsilon=0.1)

svr_regressor.fit(fw_X_train,fw_Y_train)

SVR(C=1.2, kernel='linear')

In [40]:
from sklearn.preprocessing import PolynomialFeatures
# Transform the input features to polynomial features
degree = 2 # Change this to the degree of the polynomial you want
poly_features_2 = PolynomialFeatures(degree=degree)
fw_X_train_poly_2 = poly_features_2.fit_transform(fw_X_train)
fw_X_test_poly_2 = poly_features_2.fit_transform(fw_X_test)
# Create a Linear Regression model
poly_2_regressor = LinearRegression()

# Fit the model to the polynomial features
poly_2_regressor.fit(fw_X_train_poly_2, fw_Y_train)


LinearRegression()

In [41]:
# Transform the input features to polynomial features
degree = 2 # Change this to the degree of the polynomial you want
poly_features_3 = PolynomialFeatures(degree=degree)
fw_X_train_poly_3 = poly_features_3.fit_transform(fw_X_train)
fw_X_test_poly_3 = poly_features_3.fit_transform(fw_X_test)
# Create a Linear Regression model
poly_3_regressor = LinearRegression()

# Fit the model to the polynomial features
poly_3_regressor.fit(fw_X_train_poly_3, fw_Y_train)


LinearRegression()

In [43]:
regressors=[linear_regressor,gp_regressor,svr_regressor,poly_2_regressor,poly_3_regressor]

In [44]:
mean=[]
from sklearn import metrics
for i in range(3):
    fw_Y_pred=regressors[i].predict(fw_X_test)
    mean.append(np.sqrt(metrics.mean_squared_error(fw_Y_test,fw_Y_pred)))
fw_Y_pred=regressors[3].predict(fw_X_test_poly_2)
mean.append(np.sqrt(metrics.mean_squared_error(fw_Y_test,fw_Y_pred)))
fw_Y_pred=regressors[4].predict(fw_X_test_poly_3)
mean.append(np.sqrt(metrics.mean_squared_error(fw_Y_test,fw_Y_pred)))
print(mean)

[0.5576606090094943, 0.72334694908011, 0.5627190450148725, 0.5324581891030779, 0.5324581891030779]
